# Setup

In [ ]:
# !pip install tinydb

In [99]:
import os, json
from datetime import datetime

from tinydb import TinyDB, Query

---

# Load Data

In [2]:
with open() as t1, open("data/trending2.json") as t2:
    trend1 = json.load(t1)
    trend2 = json.load(t2)

In [3]:
type(trend1)

dict

---

# Creation

##### Database Creation

In [86]:
# Create database
db = TinyDB("db_trending.json", indent=2)

# Delete any existing records in _default table
db.truncate()

# Show that db is empty
db.all()

[]

##### Table Creation

In [130]:
# Create table
table = db.table("MEGATABLE")

# Delete any existing records in MEGATABLE
table.truncate()

table.all()

[]

##### Create Custom Table Wrapper

In [127]:
import os, json
from datetime import datetime

from typing import Union

class YoutubeTable:
    def __init__(self, table):
        self.table = table
        
    def insert_videos(self, response: Union[str, dict], time: datetime = None):
        # If filepath given, convert .json at location to dict
        if isinstance(response, str):
            with open(response, "r") as file:
                response = json.load(file)
                
        # Create time string. If time not given, use current time for each videos
        if time is None:
            time = datetime.now()
        time_str = time.strftime("%m/%d/%Y_%H:%M:%S")
        
        # Insert each video individually
        for item in response["items"]:
            # Add time data to video
            entry = {"time": time_str, "timestamp": time.timestamp()}
            entry.update(item)
            
            # Convert fields prior to insertion
            entry["statistics"]["viewCount"] = int(entry["statistics"]["viewCount"])
            
            # Insert into the table
            self.table.insert(entry)

---

# Insertion

In [128]:
yt_table = YoutubeTable(table)

In [131]:
yt_table.insert_videos("data/trending1.json")

In [132]:
yt_table.insert_videos("data/trending2.json")

##### Show Contents

In [120]:
db.all()

[]

In [133]:
table.all()

[{'time': '09/14/2022_18:24:34',
  'timestamp': 1663205074.751681,
  'kind': 'youtube#video',
  'etag': 'TcTLb84_RaYhrXQF06vpsS07kQs',
  'id': 'UJ9Iz7HhU-I',
  'snippet': {'publishedAt': '2022-09-13T14:52:36Z',
   'channelId': 'UCGIY_O-8vW4rfX98KlMkvRg',
   'title': 'Nintendo Direct 9.13.2022',
   'description': 'Tune in for a Nintendo Direct livestream featuring roughly 40 minutes of information mostly focused on Nintendo Switch games launching this winter.\n\nVisit the official site: https://www.nintendo.com/nintendo-direct/09-13-2022/\n\nSubscribe for more Nintendo fun! https://goo.gl/HYYsot\r\n\r\nVisit Nintendo.com for all the latest! http://www.nintendo.com/\r\n\r\nLike Nintendo on Facebook: http://www.facebook.com/Nintendo\r\nFollow us on Twitter: http://twitter.com/NintendoAmerica\r\nFollow us on Instagram: http://instagram.com/Nintendo\r\nFollow us on Pinterest: http://pinterest.com/Nintendo',
   'thumbnails': {'default': {'url': 'https://i.ytimg.com/vi/UJ9Iz7HhU-I/default.jpg

---

# Query Examples

In [136]:
Entry = Query()  # Create query object

##### Get Nintendo Direct

In [137]:
table.search(Entry.snippet.title == "Nintendo Direct 9.13.2022")

[{'time': '09/14/2022_18:24:34',
  'timestamp': 1663205074.751681,
  'kind': 'youtube#video',
  'etag': 'TcTLb84_RaYhrXQF06vpsS07kQs',
  'id': 'UJ9Iz7HhU-I',
  'snippet': {'publishedAt': '2022-09-13T14:52:36Z',
   'channelId': 'UCGIY_O-8vW4rfX98KlMkvRg',
   'title': 'Nintendo Direct 9.13.2022',
   'description': 'Tune in for a Nintendo Direct livestream featuring roughly 40 minutes of information mostly focused on Nintendo Switch games launching this winter.\n\nVisit the official site: https://www.nintendo.com/nintendo-direct/09-13-2022/\n\nSubscribe for more Nintendo fun! https://goo.gl/HYYsot\r\n\r\nVisit Nintendo.com for all the latest! http://www.nintendo.com/\r\n\r\nLike Nintendo on Facebook: http://www.facebook.com/Nintendo\r\nFollow us on Twitter: http://twitter.com/NintendoAmerica\r\nFollow us on Instagram: http://instagram.com/Nintendo\r\nFollow us on Pinterest: http://pinterest.com/Nintendo',
   'thumbnails': {'default': {'url': 'https://i.ytimg.com/vi/UJ9Iz7HhU-I/default.jpg

##### Get Data for Views over 1,000,000

In [140]:
big_views = table.search(Entry.statistics.viewCount > 1000000)

In [143]:
print(f"{(len(big_views) / len(table.all())) * 100}% of vids are over 1,000,000 views")

42.0% of vids are over 1,000,000 views


##### Get Videos with >= / < 10 Tags

> Note: Some videos have no tags!

In [149]:
big_tags = table.search(Entry.snippet.tags.map(len) >= 10)
small_tags = table.search(Entry.snippet.tags.map(len) < 10)

In [152]:
print(f"Big tag vids: {len(big_tags)}")
for vid in big_tags:
    print(len(vid["snippet"]["tags"]), end=", ")

Big tag vids: 58
27, 21, 22, 28, 24, 21, 17, 21, 28, 24, 10, 25, 34, 18, 11, 44, 19, 22, 22, 34, 24, 10, 38, 29, 42, 27, 12, 37, 20, 19, 10, 32, 30, 19, 20, 12, 20, 28, 49, 15, 17, 17, 49, 20, 35, 22, 18, 26, 21, 23, 19, 34, 15, 28, 17, 30, 28, 15, 

In [153]:
print(f"Small tag vids: {len(small_tags)}")
for vid in small_tags:
    print(len(vid["snippet"]["tags"]), end=", ")

Small tag vids: 24
8, 9, 3, 1, 3, 6, 3, 7, 2, 4, 7, 2, 8, 9, 9, 6, 7, 9, 3, 7, 5, 5, 2, 8, 

##### Get Videos with NO Tags

In [154]:
# Use negation (~)
no_tags = table.search(~(Entry.snippet.tags.exists()))

In [155]:
print(f"Small tag vids: {len(no_tags)}")

Small tag vids: 18


In [156]:
print(len(big_tags) + len(small_tags) + len(no_tags))

100


---

# Close Database

In [18]:
db.close()